# Zettapark_Data_Engineering_Demo
这是一个非常基本的数据工程示例，演示了通过云器Zettapark Python代码进行读取、分组和写入数据等基本DataFrame操作。本示例使用云器Lakehouse内置的免费示例数据库（clickzetta_sample_data.tpch_100g）作为数据源。

步骤很简单：

1，通过Zettapark连接到云器Lakehouse

2，通过SupplierKey连接 2个大型表 (LINEITEMS 有 6 亿行 & SUPPLIER 有 100 万行)

3，通过将虚拟计算集群调整到XSMALL、MEDIUM不同规格来演示按需扩展

4，对比不同规格的虚拟计算集群对同一个任务的执行时间的不同

    汇总供应商和零件编号的数据以计算总和、最小值和最大值（3500万行）
    
    将结果数据框写入云器Lakehouse物理表（8000万行）

**整个操作从调整计算资源、读取数据、连接、汇总大约需要100来秒，向您展示了云器Lakehouse强大的功能和即时可扩展性和性能。**

## 安装云器Zettapark

In [1]:
# !pip install clickzetta-zettapark-python



---



## 通过ZettaPark连接到云器Lakehouse(& without PySpark)

In [2]:
import time
from clickzetta.zettapark.session import Session
import clickzetta.zettapark.functions as f
from clickzetta.zettapark import Session, DataFrame
from clickzetta.zettapark.functions import udf, col
from clickzetta.zettapark.types import IntegerType
from clickzetta.zettapark.functions import call_udf


# <----- Make these changes before running the notebook -------
# Change Connection params to match your environment
# <----------------------------------------------------------------------------

VCLUSTER_Name = 'default_ap'
VCLUSTER_Size = "XSMALL"
VCLUSTER_ReSize = "MEDIUM"
Workspace_Name = 'gharchive'
Schema_Name = 'Public'

In [3]:
import json
from clickzetta.zettapark.session import Session

# 从配置文件中读取参数
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

print("正在连接到云器Lakehouse.....\n")

# 创建会话
session = Session.builder.configs(config).create()

print("连接成功！...\n")


正在连接到云器Lakehouse.....

连接成功！...



In [4]:
sql_cmd = f"CREATE  VCLUSTER IF NOT EXISTS {VCLUSTER_Name} VCLUSTER_SIZE = {VCLUSTER_Size}  AUTO_SUSPEND_IN_SECOND = 10 "
print("XSMALL VCLUSTER 创建就绪 \n")

session.sql(sql_cmd).collect() 


session.use_database(Workspace_Name)
session.use_schema(Schema_Name)
session.use_warehouse(VCLUSTER_Name)

XSMALL VCLUSTER 创建就绪 



## 开始数据工程Data Engineering Process

In [5]:
from clickzetta.zettapark.functions import col, sum, min, max

print("Joining, Aggregating with 2 large tables(600M & 1M rows) & Writing results to new table(80M rows) ..\n")

dfLineItems = session.table("clickzetta_sample_data.tpch_100g.LINEITEM")  # 600 Million Rows
dfSuppliers = session.table("clickzetta_sample_data.tpch_100g.SUPPLIER")  # 1 Million Rows

print('Lineitems Table: %s 行' % dfLineItems.count())
print('Suppliers Table: %s 行' % dfSuppliers.count())

# 3 - JOIN TABLES
dfJoinTables = dfLineItems.join(dfSuppliers, dfLineItems["L_SUPPKEY"] == dfSuppliers["S_SUPPKEY"])

# 4 - SUMMARIZE THE DATA BY SUPPLIER, PART, SUM, MIN & MAX
dfSummary = dfJoinTables.groupBy("S_NAME", "L_PARTKEY").agg(
    sum(col("L_QUANTITY")).alias("TOTAL_QTY"),
    min(col("L_QUANTITY")).alias("MIN_QTY"),
    max(col("L_QUANTITY")).alias("MAX_QTY")
)

dfSummary.show()


Joining, Aggregating with 2 large tables(600M & 1M rows) & Writing results to new table(80M rows) ..

Lineitems Table: 600037902 行
Suppliers Table: 1000000 行
------------------------------------------------------------------
|s_name              |l_partkey  |total_qty  |min_qty  |max_qty  |
------------------------------------------------------------------
|Supplier#000260113  |19260112   |186.00     |7.00     |45.00    |
|Supplier#000043870  |4043869    |171.00     |6.00     |50.00    |
|Supplier#000168305  |4668296    |151.00     |16.00    |38.00    |
|Supplier#000433333  |11183321   |228.00     |1.00     |49.00    |
|Supplier#000869920  |18369883   |217.00     |1.00     |46.00    |
|Supplier#000648233  |2398230    |275.00     |2.00     |49.00    |
|Supplier#000405848  |10155837   |335.00     |2.00     |49.00    |
|Supplier#000569567  |3819557    |143.00     |3.00     |37.00    |
|Supplier#000501244  |13001217   |228.00     |7.00     |49.00    |
|Supplier#000129062  |8129061    |254.

In [6]:
# 2 - READ & JOIN 2 LARGE TABLES (600M & 1M rows)
from clickzetta.zettapark.functions import col, sum, min, max

print("正在合并和聚合两个大表（6亿行和100万行），并将结果写入新表（8000万行）..\\n")

dfLineItems = session.table("clickzetta_sample_data.tpch_100g.LINEITEM")  # 600 Million Rows
dfSuppliers = session.table("clickzetta_sample_data.tpch_100g.SUPPLIER")  # 1 Million Rows

print('Lineitems Table: %s 行' % dfLineItems.count())
print('Suppliers Table: %s 行' % dfSuppliers.count())

# 3 - JOIN TABLES
dfJoinTables = dfLineItems.join(dfSuppliers, dfLineItems["L_SUPPKEY"] == dfSuppliers["S_SUPPKEY"])

# 4 - SUMMARIZE THE DATA BY SUPPLIER, PART, SUM, MIN & MAX
dfSummary = dfJoinTables.groupBy("S_NAME", "L_PARTKEY").agg(
    sum("L_QUANTITY").alias("TOTAL_QTY"),
    min("L_QUANTITY").alias("MIN_QTY"),
    max("L_QUANTITY").alias("MAX_QTY")
)

dfSummary.show()

正在合并和聚合两个大表（6亿行和100万行），并将结果写入新表（8000万行）..\n
Lineitems Table: 600037902 行
Suppliers Table: 1000000 行
------------------------------------------------------------------
|s_name              |l_partkey  |total_qty  |min_qty  |max_qty  |
------------------------------------------------------------------
|Supplier#000175476  |7675461    |228.00     |6.00     |50.00    |
|Supplier#000014993  |6514980    |206.00     |11.00    |50.00    |
|Supplier#000265045  |9515017    |121.00     |3.00     |35.00    |
|Supplier#000177824  |3427814    |239.00     |6.00     |45.00    |
|Supplier#000800661  |13800660   |402.00     |4.00     |49.00    |
|Supplier#000839196  |2589193    |287.00     |1.00     |45.00    |
|Supplier#000004061  |17754043   |281.00     |10.00    |49.00    |
|Supplier#000598522  |1098519    |175.00     |3.00     |42.00    |
|Supplier#000892668  |16392635   |179.00     |1.00     |46.00    |
|Supplier#000660953  |4160944    |336.00     |3.00     |47.00    |
-----------------------------

## 3. 通过不同计算资源（虚拟集群）完成同样的计算任务需要不同的时间，查看弹性扩缩的效果.

In [7]:
start_time = time.time()

# 4 - 将虚拟计算集群大小调整为 XSMALL
print(f"正在调整到 {VCLUSTER_Size} ..")

sql_cmd = f"ALTER VCLUSTER {VCLUSTER_Name} SET VCLUSTER_SIZE = '{VCLUSTER_Size}' "
session.sql(sql_cmd).collect()  

print("完成！...\n\n")


# 5 - 将结果写入新表（8000万行）
# <-- 这是当所有之前的操作编译并作为单个作业执行时
print("正在创建目标 SALES_SUMMARY 表...\n\n")
dfSummary.write.mode("overwrite").saveAsTable("SALES_SUMMARY")
print("目标表已创建！...")

# 6 - 查询结果（8000万行）
print("正在查询结果...\n")
dfSales = session.table("SALES_SUMMARY")
dfSales.show()
end_time = time.time()

print("--- 连接、汇总和写入结果到新表用了 %s 秒 --- \n" % int(end_time - start_time))
print("--- 向 SALES_SUMMARY 表写入了 %s 行" % dfSales.count())

# 7 - 将虚拟计算集群大小减少到 XSMALL
print("将 VCLUSTER 缩小到 XS...\n")
sql_cmd = "ALTER VCLUSTER {} SET VCLUSTER_SIZE = 'XSMALL'".format(VCLUSTER_Name)
session.sql(sql_cmd).collect()  

print("完成！...\n")


正在调整到 XSMALL ..
完成！...


正在创建目标 SALES_SUMMARY 表...


目标表已创建！...
正在查询结果...

------------------------------------------------------------------
|s_name              |l_partkey  |total_qty  |min_qty  |max_qty  |
------------------------------------------------------------------
|Supplier#000528338  |16778289   |255.00     |5.00     |46.00    |
|Supplier#000722723  |5722722    |361.00     |3.00     |50.00    |
|Supplier#000856402  |18106347   |206.00     |8.00     |50.00    |
|Supplier#000688944  |17938892   |235.00     |2.00     |49.00    |
|Supplier#000235253  |9235252    |135.00     |8.00     |49.00    |
|Supplier#000807610  |6557603    |168.00     |2.00     |47.00    |
|Supplier#000840619  |17590601   |171.00     |3.00     |42.00    |
|Supplier#000630379  |5880363    |171.00     |5.00     |48.00    |
|Supplier#000060992  |19810972   |98.00      |3.00     |37.00    |
|Supplier#000605358  |19855300   |218.00     |11.00    |50.00    |
------------------------------------------------------

In [8]:
start_time = time.time()

# 4 - 将虚拟计算集群大小增加到 MEDIUM
print(f"正在将 {VCLUSTER_Size} 调整为 {VCLUSTER_ReSize} ..")

sql_cmd = f"ALTER VCLUSTER {VCLUSTER_Name} SET VCLUSTER_SIZE = '{VCLUSTER_ReSize}'"
session.sql(sql_cmd).collect()  

print("完成！...\n\n")


# 5 - 将结果写入新表（8000万行）
# <-- 这是当所有之前的操作编译并作为单个作业执行时
print("正在创建目标 SALES_SUMMARY 表...\n\n")
dfSummary.write.mode("overwrite").saveAsTable("SALES_SUMMARY")
print("目标表已创建！...")

# 6 - 查询结果（8000万行）
print("正在查询结果...\n")
dfSales = session.table("SALES_SUMMARY")
dfSales.show()
end_time = time.time()
print("--- 连接、汇总和写入结果到新表用了 %s 秒 --- \n" % int(end_time - start_time))
print("--- 向 SALES_SUMMARY 表写入了 %s 行" % dfSales.count())

# 7 - 将虚拟计算集群大小少到 XSMALL
print("将 VCLUSTER 缩小到 XSMALL...\n")
sql_cmd = f"ALTER VCLUSTER {VCLUSTER_Name} SET VCLUSTER_SIZE = {VCLUSTER_Size}"
session.sql(sql_cmd).collect()  

print("完成！...\n")


正在将 XSMALL 调整为 MEDIUM ..
完成！...


正在创建目标 SALES_SUMMARY 表...


目标表已创建！...
正在查询结果...

------------------------------------------------------------------
|s_name              |l_partkey  |total_qty  |min_qty  |max_qty  |
------------------------------------------------------------------
|Supplier#000315647  |19565589   |202.00     |6.00     |50.00    |
|Supplier#000973017  |19973016   |140.00     |9.00     |48.00    |
|Supplier#000019398  |7019397    |322.00     |4.00     |50.00    |
|Supplier#000905072  |8655063    |185.00     |23.00    |49.00    |
|Supplier#000481081  |8481080    |229.00     |1.00     |44.00    |
|Supplier#000378908  |9628880    |149.00     |1.00     |36.00    |
|Supplier#000233092  |15483046   |161.00     |5.00     |50.00    |
|Supplier#000773461  |8023436    |413.00     |1.00     |49.00    |
|Supplier#000601423  |3351419    |204.00     |5.00     |48.00    |
|Supplier#000948894  |16198845   |284.00     |20.00    |50.00    |
---------------------------------------------

# **Zettapark 相对于 Spark 和 PySpark 的优势**
- **迁移快捷** 因为代码基本相同，不需要重新学习新语言
- **更便宜** 因为计算完全Severless化。它可以秒级扩展（向上/向下），并且仅在使用时运行（产生成本）。
- **秒级即时扩缩** 同样的计算任务，XSMALL规格虚拟计算集群VCluster需要75秒左右，而MEDIUM规格的只需要20秒。
- **更快** 因为消除了所有不必要的数据移动 = 使用计算的时间更少 = 成本更低
- **更易使用** = 更少的人力 因为计算和存储几乎不需要维护。